In [1]:
import json
import re
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
def vinted_session():
    s = cloudscraper.create_scraper()
    s.headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36',
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'fr',
        'DNT': '1',
        'Connection': 'keep-alive',
        'TE': 'Trailers',
    }
    req = s.get("https://www.vinted.fr/")
    csrf_token_index = req.text.find("CSRF_TOKEN")
    start_index = csrf_token_index + len("CSRF_TOKEN\":\"")
    end_index = req.text.find("\",", start_index)
    csrf_token_value = req.text[start_index:end_index]


    s.headers['X-CSRF-Token'] = csrf_token_value
    return s

In [3]:
def scrapping_user(s, idUser, annonce_id):

    item_titles_advice, ratings_advice, feedbacks_advice = [], [], []
    item_id_dressing, item_title_dressing, item_brand_dressing = [], [], []
    item_size_dressing, item_price_dressing, item_created_dressing,item_description_dressing = [], [], [],[]

    try:
        url = f"https://www.vinted.fr/api/v2/users/{idUser}?localize=false"
        check = s.get(url)
        check.raise_for_status()
        json_data = json.loads(check.text)

        if json_data['user']['photo'] is None:
            photo = 'Null'
        else:
            photo = json_data['user']['photo']['url']
        
        if json_data['user']['business_account'] is None:
            business=False
        else:
            business=True

        if json_data['user']['feedback_count'] > 0:
            url_advice = f"https://www.vinted.fr/api/v2/user_feedbacks?user_id={idUser}&page=1&per_page={json_data['user']['feedback_count']}&by=all"
            check_advice = s.get(url_advice)
            check_advice.raise_for_status()
            json_data_advice = json.loads(check_advice.text)
            for advice in json_data_advice['user_feedbacks']:
                item_titles_advice.append(advice['item_title'])
                ratings_advice.append(advice['rating'])
                feedbacks_advice.append(advice['feedback'])
            time.sleep(1)  # Attendre 1 seconde avant le prochain appel API

        url_dressing = f"https://www.vinted.fr/api/v2/users/{idUser}/items?page=1&per_page={json_data['user']['item_count']}&order=relevance"
        check_dressing = s.get(url_dressing)
        check_dressing.raise_for_status()
        json_data_dressing = json.loads(check_dressing.text)

        for item in json_data_dressing["items"]:
            if item['id'] == annonce_id:
                annonce_date = item['created_at_ts']
                annonce_description = item['description']
            else:
                item_id_dressing.append(item['id'])
                item_title_dressing.append(item['title'])
                item_brand_dressing.append(item['brand'])
                item_size_dressing.append(item['size'])
                item_price_dressing.append(item['original_price_numeric'])
                item_created_dressing.append(item['created_at_ts'])
                item_description_dressing.append(item['description'])
        time.sleep(1)  # Attendre 1 seconde avant le prochain appel API

        

        return (json_data['user']['login'], json_data['user']['email'], json_data['user']['gender'], json_data['user']['item_count'],
                  json_data['user']['followers_count'], json_data['user']['following_count'],
                  json_data['user']['feedback_count'], json_data['user']['positive_feedback_count'],
                  json_data['user']['neutral_feedback_count'], json_data['user']['negative_feedback_count'],
                  json_data['user']['account_ban_date'], json_data['user']['is_account_ban_permanent'],
                  json_data['user']['is_publish_photos_agreed'], json_data['user']['expose_location'],
                  json_data['user']['city'], json_data['user']['country_title'], business,
                  json_data['user']['total_items_count'], json_data['user']['verification']['email']['valid'],
                  json_data['user']['verification']['facebook']['valid'],
                  json_data['user']['verification']['google']['valid'],
                  json_data['user']['verification']['phone']['valid'], json_data['user']['is_hated'],
                  json_data['user']['hates_you'], json_data['user']['can_view_profile'], item_titles_advice,
                  ratings_advice, feedbacks_advice, item_id_dressing, item_title_dressing, item_brand_dressing,
                  item_size_dressing, item_price_dressing, item_created_dressing, item_description_dressing, photo, annonce_date, annonce_description)

    except Exception as e:
        print(f"Une erreur s'est produite : {e}")
        return None


In [4]:
def scrapping_annonces(dataset_final, legend, s):
    try:
        url = f"https://www.vinted.fr/api/v2/catalog/items?page=1&per_page=10&search_text={legend}&catalog_ids=&color_ids=&brand_ids=&size_ids=&material_ids=&video_game_rating_ids=&status_ids="
        check = s.get(url)
        check.raise_for_status()
        json_data = json.loads(check.text)

        for annonce in json_data['items']:
            try:
                
                (login, email, gender, item_count, followers_count, following_count,
                feedback_count, positive_feedback_count, neutral_feedback_count,
                negative_feedback_count, account_ban_date, is_account_ban_permanent,
                is_publish_photos_agreed, expose_location, city, country_title, business_account,
                total_items_count, verification_email, verification_facebook, verification_google,
                verification_phone, is_hated, hates_you, can_view_profile, item_titles_advice,
                ratings_advice, feedbacks_advice, item_id_dressing, item_title_dressing,
                item_brand_dressing, item_size_dressing, item_price_dressing, item_created_dressing,
                item_description_dressing, photo, dateA, descriptionA) = scrapping_user(s, annonce['user']['id'], annonce['id'])
                dataset_annonce = pd.DataFrame(
                    [[annonce['id'], annonce['title'], annonce['price'], annonce['brand_title'], annonce['status'],
                      annonce['size_title'], annonce['favourite_count'], annonce['photo']['url'], dateA, descriptionA,
                      annonce['user']['id'],login, email, gender, item_count, followers_count, following_count,
                      feedback_count, positive_feedback_count, neutral_feedback_count,
                      negative_feedback_count, account_ban_date, is_account_ban_permanent,
                      is_publish_photos_agreed, expose_location, city, country_title, business_account,
                      total_items_count, verification_email, verification_facebook, verification_google,
                      verification_phone, is_hated, hates_you, can_view_profile, item_titles_advice,
                      ratings_advice, feedbacks_advice, item_id_dressing, item_title_dressing,
                      item_brand_dressing, item_size_dressing, item_price_dressing, item_created_dressing,
                      item_description_dressing, photo]],
                    columns=['idA', 'titleA', 'price', 'brand', 'etat', 'taille', 'favourite_count', 'photoA', 'dateA',
                             'descriptionA', 'idUser','login','email', 'gender', 'item_count', 'followers_count', 'following_count',
                     'feedback_count', 'positive_feedback_count', 'neutral_feedback_count',
                     'negative_feedback_count', 'account_ban_date', 'is_account_ban_permanent',
                     'is_publish_photos_agreed', 'expose_location', 'city', 'country_title', 'business_account',
                     'total_items_count', 'verification_email', 'verification_facebook', 'verification_google',
                     'verification_phone', 'is_hated', 'hates_you', 'can_view_profile', 'item_titles_advice',
                     'ratings_advice', 'feedbacks_advice', 'item_id_dressing', 'item_title_dressing',
                     'item_brand_dressing', 'item_size_dressing', 'item_price_dressing', 'item_created_dressing',
                     'item_description_dressing', 'photo']
                )
                dataset_final = dataset_final.append(dataset_annonce, ignore_index=True)
            except Exception as e_annonce:
                print(f"Une erreur s'est produite lors du scraping d'une annonce : {e_annonce}")
            
            time.sleep(1)  # Attendre 1 seconde avant le prochain appel API pour les annonces
        
        
        return dataset_final

    except Exception as e:
        print(f"Une erreur s'est produite lors du scraping des annonces : {e}")
        return dataset_final

In [5]:
dataset_final=pd.DataFrame(columns=['idA','titleA','price','brand','etat','taille','favourite_count','photoA','idUser','login','email', 'gender', 'item_count', 'followers_count', 'following_count',
                     'feedback_count', 'positive_feedback_count', 'neutral_feedback_count',
                     'negative_feedback_count', 'account_ban_date', 'is_account_ban_permanent',
                     'is_publish_photos_agreed', 'expose_location', 'city', 'country_title', 'business_account',
                     'total_items_count', 'verification_email', 'verification_facebook', 'verification_google',
                     'verification_phone', 'is_hated', 'hates_you', 'can_view_profile', 'item_titles_advice',
                     'ratings_advice', 'feedbacks_advice', 'item_id_dressing', 'item_title_dressing',
                     'item_brand_dressing', 'item_size_dressing', 'item_price_dressing', 'item_created_dressing',
                     'item_description_dressing', 'photo'])

In [6]:
s=vinted_session()
legend='Nike+dunk+low+pandas'
dataset_final=scrapping_annonces(dataset_final,legend,s)

In [7]:
dataset_final.head()

,idA,titleA,price,brand,etat,taille,favourite_count,photoA,idUser,login,...,item_id_dressing,item_title_dressing,item_brand_dressing,item_size_dressing,item_price_dressing,item_created_dressing,item_description_dressing,photo,dateA,descriptionA
0,3907000764,Dunk low 31,14.0,Nike,Bon état,31,13,https://images1.vinted.net/t/01_0033c_oe45Ebpy...,14401341,lora2600,...,"[2177376513, 3907790024, 3905674659, 390507492...","[Combinaison pilote Chloé , Pantalon de ski fi...","[Chloé, McKinley, Clementoni, K-Way, Zara, Uni...","[9-12 mois / 74 cm, 8 ans / 128 cm, Taille uni...","[100.0, 40.0, 5.0, 80.0, 9.0, 10.0, 12.0, 110....","[2022-09-08T10:54:51+02:00, 2023-12-25T22:34:2...",[Combi pilote molletonnée et bien chaude\nExce...,https://images1.vinted.net/t/02_008a4_7sqpzVRX...,2023-12-25T14:15:04+01:00,Pointure 31 \nNike Dunk low panda ( noir et bl...
1,3910185679,Nïkế🐼Dưnk Black Loẘ White Pandä 40,46.0,Nike,Neuf sans étiquette,40,0,https://images1.vinted.net/t/01_00f82_qCSnjBTL...,179251322,gurctdgxqcbzzwtn,...,[],[],[],[],[],[],[],Null,2023-12-26T22:10:45+01:00,Nïkế🐼Dưnk Black Loẘ White Pandä 40
2,3910261878,Nïkế🐼Dưnk Black Loẘ White Pandä,32.55,PLEASE,Neuf avec étiquette,37.5,1,https://images1.vinted.net/t/03_01dd4_573SrgER...,178782909,fdkdqqcxfwp,...,"[3910263151, 3910262687, 3910262288, 391026149...","[Nïkế🐼Dưnk Black Loẘ White Pandä, Nïkế🐼Dưnk Bl...","[PLEASE, PLEASE, PLEASE, PLEASE, PLEASE, PLEASE]","[42, 43, 40, 42, 41, 38]","[32.55, 32.55, 32.55, 32.55, 32.55, 32.55]","[2023-12-26T22:51:34+01:00, 2023-12-26T22:51:1...","[Nïkế🐼Dưnk Black Loẘ White Pandä, Nïkế🐼Dưnk Bl...",https://images1.vinted.net/t/03_00f19_Zqob6eSK...,2023-12-26T22:50:47+01:00,Nïkế🐼Dưnk Black Loẘ White Pandä
3,3908296106,Nike Dunk🎁 Low Retro🎁 White Black Panda EU:39,31.0,Nike,Bon état,39,46,https://images1.vinted.net/t/02_00649_R9vthnkk...,145800688,nicolebeaujoliezz,...,"[3909942276, 3908218266]",[Nike Dunk🎁 Low Retro🎁 White Black Panda EU:38...,"[Nike, ]","[38, S / 36 / 8]","[31.0, 15.0]","[2023-12-26T20:28:28+01:00, 2023-12-26T11:16:3...",[Nike Dunk🎁 Low Retro🎁 White Black Panda EU:38...,https://images1.vinted.net/t/01_01fa2_y6kvvRMY...,2023-12-26T11:43:30+01:00,Nike Dunk🎁 Low Retro🎁 White Black Panda EU:39\n
4,3910162233,Nïkế🐼Dưnk Black Loẘ White Pandä 40,46.5,Nike,Neuf avec étiquette,40,0,https://images1.vinted.net/t/02_0248e_292hMY79...,179250841,aflroirf,...,"[3910164137, 3910163059]","[Nïkế🐼Dưnk Black Loẘ White Pandä 40, Nïkế🐼Dưnk...","[Nike, Nike]","[40, 40]","[46.5, 46.0]","[2023-12-26T14:00:44-07:00, 2023-12-26T22:00:1...","[Nïkế🐼Dưnk Black Loẘ White Pandä 40, Nïkế🐼Dưnk...",Null,2023-12-26T21:59:48+01:00,Nïkế🐼Dưnk Black Loẘ White Pandä 40


In [12]:
dataset_final['item_titles_advice']

0    [Jeu rush jour jr , Doudoune style cuir Zadig ...
1                                                   []
2                                                   []
3                                                   []
4                                                   []
5                                                   []
6                                                   []
7                                                   []
8                                                   []
9                                                   []
Name: item_titles_advice, dtype: object

In [13]:
mots_arnaque = ['arnaque', 'contrefaçon', 'imitation', 'fausse', 'non authentique', 'replica', 'copie', 'faux logo', 'faussaire', 'frauduleux',
                'fausse marque', 'knockoff', 'réplique', 'contrefaite', 'pas authentique', 'non authentique', 'fausse version', 'non originale',
                'imitation de marque', 'fausse représentation', 'non conforme', 'trompeur', 'escroquerie', 'fraude', 'malhonnête', 
                'tromperie', 'duperie', 'faux', 'illégitime', 'manipulation', 'supercherie', 'fourberie', 'ruse', 'mensonge', 
                'tricherie', 'infraction', 'désinformation', 'trafic', 'manoeuvre frauduleuse', 'corruption', 'hameçonnage', 'usurpation d\'identité']

mots_arnaque_sans_accents = [unidecode.unidecode(mot) for mot in mots_arnaque]

# Fusionner les deux listes
mots_arnaque.extend(mots_arnaque_sans_accents)

def arnaque_advice(row):
    for i in row['']

0    [jeu en parfait état mais boîte en très mauvai...
1                                                   []
2                                                   []
3                                                   []
4                                                   []
5                                                   []
6                                                   []
7                                                   []
8                                                   []
9                                                   []
Name: feedbacks_advice, dtype: object

In [35]:
from unidecode import unidecode

def frauduleux_advice(row):
    mots_arnaque = ['arnaque', 'contrefaçon', 'imitation', 'fausse', 'non authentique', 'replica', 'copie', 'faux logo', 'faussaire', 'frauduleux',
                'fausse marque', 'knockoff', 'réplique', 'contrefaite', 'pas authentique', 'non authentique', 'fausse version', 'non originale',
                'imitation de marque', 'fausse représentation', 'non conforme', 'trompeur', 'escroquerie', 'fraude', 'malhonnête', 
                'tromperie', 'duperie', 'faux', 'illégitime', 'manipulation', 'supercherie', 'fourberie', 'ruse', 'mensonge', 
                'tricherie', 'infraction', 'désinformation', 'trafic', 'manoeuvre frauduleuse', 'corruption', 'hameçonnage', 'usurpation d\'identité']

    mots_arnaque_sans_accents = [unidecode(mot) for mot in mots_arnaque]
    mots_arnaque.extend(mots_arnaque_sans_accents) 
    i=0
    for advice in row['ratings_advice']:
        if advice<=3:
            for word in row['feedbacks_advice'][i].lower():
                if word in mots_arnaque:
                    print(word)
                    return True
        i+=1
    return False

for index, row in dataset_final.iterrows():
    frauduleux_advice(row)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [ ]:
modeles_similaires = ['nocta','sacai','protection pack', 'air jordan 1','air jordan 4','air jordan 3','air jordan one','air jordan four','air jordan three',
                      'jordan 4','jordan 3','jordan 1', 'jordan one','jordan four','jordan three', 'yeezy','off-white', 'travis scott',
                        'supreme','virgil abloh', 'fragment', 'jordan low', 'campus','samba']

si d'autres dunk si le meme modele, si paire tendance en neuf


In [34]:
len(dataset_final['ratings_advice'][0])

100

title:Nïkế🐼 beaucoup de truc comme ça
brand: pas que nike beaucoup novo
favourite_count: ne pas prendre pas logique
description=title

email : null 

account_ban_date   inutile
is_account_ban_permanent   inutile
is_publish_photos_agreed 

is_hated                   50 non-null     object
 33  hates_you
 can_view_profile
